# Libraries

In [ ]:
import numpy as np
import pandas as pd
import math
from typing import Callable, Optional
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy import stats

from scipy.optimize import fsolve, brentq, minimize
from scipy.special import logsumexp, gamma as Gamma, gammainc
from scipy.stats import weibull_min

from tqdm import tqdm

# Import RA-UCB functions from external module
from Utils_weibull import (
    oracle_weibull_softmax,
    g_weibull,
    g_inv_weibull,
    compute_confidence_bounds_weibull_vec,
    select_allocation,
    update_estimates_weibull,
    compute_regret,
    simulate_one_round_weibull,
    sanity_check,
)

# EdNet KT3

In [ ]:
df = pd.read_csv("./pseudo_users_dataset.csv")  

# Robust boolean conversion + time cleaning
df["is_correct"] = df["is_correct"].astype(str).str.lower().isin(["true","1","yes","y"])
df["response_time_ms"] = pd.to_numeric(df["response_time_ms"], errors="coerce")
df["response_time_ms"] = df["response_time_ms"] / 1000
df = df.dropna(subset=["question_id","response_time_ms","is_correct"])
df = df[df["response_time_ms"] > 0]

res = (df.groupby("question_id")
         .apply(lambda g: pd.Series({
             "n": len(g),
             "p_i": g["is_correct"].mean(),
             "K_i": weibull_min.fit(g["response_time_ms"].values, floc=0)[0],      # shape
             "scale_i": weibull_min.fit(g["response_time_ms"].values, floc=0)[2]  # scale (λ = 1/scale)
         }))
         .reset_index()
      )

res

# Experiments

## Pseudo syntetic on 10 first arms

In [ ]:

K = 10
B = 700
D = 1
T = 5000
method="trust-constr"
n_restarts=5


p_min, p_max = 0.01, 1.0
lambda_min, lambda_max = 1/B, 80/B
k_min, k_max = 0.8, 2.5


np.random.seed(42)
p_true = np.array([0.3038,0.3338,0.5020,0.4550,0.2208,0.5988,0.7194,0.6264,0.6572,0.4288])
lambda_true = np.array([1/35.804217,1/39.764151,1/43.488183,1/35.113810,1/48.429560,1/37.517613,1/36.014443,1/39.964080,1/38.084058,1/40.805778])
k_vec = np.array([1.832726,1.902398,1.604154,1.938767,1.197973,1.614411,1.299792,1.400885,1.479928,1.536946])  

print("True parameters (unknown to the algorithm):")
print("p_true =", p_true)
print("lambda_true =", lambda_true)
print("k_vec (connu) =", k_vec)

#Optimal allocation
x_star = oracle_weibull_softmax(p_true, lambda_true, lambda_true, B, k_vec, n_restarts=5, seed=0)
print("\nx_star =", x_star)
print("sum(x_star) =", np.sum(x_star))


t1 = 0  # Reward time
beta = 0.001
init_steps_per_arm = int(beta * T / K)

#Variable estimation
n = np.zeros((int(T/K), K))
Szk = np.zeros((int(T/K), K))
theta_hat = np.zeros((int(T/K), K))
lambda_hat = np.zeros((int(T/K), K))
p_hat = np.zeros((int(T/K), K))
x_est = np.zeros((int(T/K), K))

# Reward Variables
x_reward = np.zeros((T, K))
X = np.zeros((T, K))  # Latent response times
Y = np.zeros((T, K))  # Results (correct/incorrect)
F = np.zeros((T, K))
f = np.zeros((T, K))
f_star = np.zeros((T, K))
reward = np.zeros(T)
opt_reward = np.zeros(T)
regret = np.zeros(T)
cum_regret = np.zeros(T)

phi = np.zeros((int(T/K), K))
psi = np.zeros((int(T/K), K))

# INITIALIZATION PHASE
print("\nInitialization phase...")
for i in range(K):
    for step in range(init_steps_per_arm):
        x_reward[t1 + 1] = np.zeros(K)
        x_reward[t1 + 1, i] = B

        
        Y_sample = np.random.binomial(n=1, p=p_true)
        U = np.random.uniform(0, 1, size=K)
        X_sample = (-np.log(1 - U)) ** (1.0 / k_vec) / lambda_true
        
        X[t1 + 1] = X_sample
        Y[t1 + 1] = Y_sample

        for k in range(K):
            if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                f[t1 + 1, k] = 1
                F[t1 + 1, k] = X_sample[k]
            else:
                f[t1 + 1, k] = 0
                F[t1 + 1, k] = 10 * B
                
            if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                f_star[t1 + 1, k] = 1
            else:
                f_star[t1 + 1, k] = 0

        phi[t1 // K + 1, i] = f[t1 + 1, i]
        psi[t1 // K + 1, i] = F[t1 + 1, i]
        update_estimates_weibull(t1 // K, i, k_vec[i], phi, psi, n, Szk, theta_hat, lambda_hat, p_hat, x_est, B)
        compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)
        t1 += 1

# MAIN PHASE 
print("Main phase...")
total_iterations = (T // K - 1 - t1 // K) * K
with tqdm(total=total_iterations, desc="Progression") as pbar:
    for t in range(t1 // K, T // K - 1):  # Estimation time
        for i in range(K):
            
            Y_sample = np.random.binomial(n=1, p=p_true)
            U = np.random.uniform(0, 1, size=K)
            X_sample = (-np.log(1 - U)) ** (1.0 / k_vec) / lambda_true

            
            p_bar, lambda_bar, lambda_bar_prime = compute_confidence_bounds_weibull_vec(
                t, i, p_hat, lambda_hat, n, x_est, B, D, K, k_vec)
            
            
            x_reward[t1 + 1] = select_allocation(lambda_bar, lambda_bar_prime, p_bar, B, k_vec)
            x_est[t + 1, i] = x_reward[t1 + 1, i].copy()

            X[t1 + 1] = X_sample
            Y[t1 + 1] = Y_sample

            for k in range(K):
                if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                    f[t1 + 1, k] = 1
                    F[t1 + 1, k] = X_sample[k]
                else:
                    f[t1 + 1, k] = 0
                    F[t1 + 1, k] = 10 * B
                    
                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star[t1 + 1, k] = 1
                else:
                    f_star[t1 + 1, k] = 0

            phi[t + 1, i] = f[t1 + 1, i]
            psi[t + 1, i] = F[t1 + 1, i]
            update_estimates_weibull(t, i, k_vec[i], phi, psi, n, Szk, theta_hat, lambda_hat, p_hat, x_est, B)
            compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)

            t1 += 1
            pbar.update(1)

print(f"\nSimulation complete!")

actual_cum_regret = np.sum(regret[1:t1+1])
print(f"Final cumulative regret (correct): {actual_cum_regret:.2f}")
print(f"Last index t1 = {t1}")

print("\n=== DEBUG INFO ===")
print(f"Total reward: {np.sum(reward[1:t1+1]):.2f}")
print(f"Total opt_reward: {np.sum(opt_reward[1:t1+1]):.2f}")
print(f"Mean instantaneous regret: {np.mean(regret[1:t1+1]):.4f}")
print(f"Std instantaneous regret: {np.std(regret[1:t1+1]):.4f}")
print(f"Min/Max instantaneous regret: {np.min(regret[1:t1+1]):.2f} / {np.max(regret[1:t1+1]):.2f}")


print("\n=== ALLOCATION COMPARISON (last round) ===")
print(f"x_star = {x_star}")
print(f"x_reward (last valid) = {x_reward[t1]}")
print(f"Allocation error (L2): {np.linalg.norm(x_reward[t1] - x_star):.4f}")


fig, axes = plt.subplots(2, 2, figsize=(14, 10))


cum_regret_correct = np.cumsum(regret[1:t1+1])

# Cumulative regret
ax1 = axes[0, 0]
t_axis = np.arange(1, t1+1)
ax1.plot(t_axis, cum_regret_correct, linewidth=2, label='Cumulative regret')
ax1.set_xlabel('t')
ax1.set_ylabel('Cumulative regret')
ax1.set_title('Cumulative regret of RA-UCB Weibull')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2) instant regret
ax2 = axes[0, 1]
window = 100
regret_smooth = np.convolve(regret[1:t1+1], np.ones(window)/window, mode='valid')
ax2.plot(np.arange(len(regret_smooth)), regret_smooth, linewidth=1, label=f'Instantaneous regret (MA {window})')
ax2.axhline(0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('t')
ax2.set_ylabel('Regret instantané')
ax2.set_title('Instantaneous regret (moving average)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3) per round reward
ax3 = axes[1, 0]
reward_smooth = np.convolve(reward[1:t1+1], np.ones(window)/window, mode='valid')
opt_reward_smooth = np.convolve(opt_reward[1:t1+1], np.ones(window)/window, mode='valid')
ax3.plot(np.arange(len(reward_smooth)), reward_smooth, label='Reward (algorithm)')
ax3.plot(np.arange(len(opt_reward_smooth)), opt_reward_smooth, label='Reward (oracle)', linestyle='--')
ax3.set_xlabel('t')
ax3.set_ylabel('Reward')
ax3.set_title('Mean reward per round')
ax3.legend()

ax4 = axes[1, 1]
arm_to_track = 1 
ax4.plot(np.arange(1, t1+1), x_reward[1:t1+1, arm_to_track], label=f'x_reward[arm {arm_to_track}]', alpha=0.7)
ax4.axhline(x_star[arm_to_track], color='r', linestyle='--', label=f'x_star[arm {arm_to_track}]')
ax4.set_xlabel('t')
ax4.set_ylabel('Allocation')
ax4.set_title(f'Allocation to arm {arm_to_track} vs optimal')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## On real data : user revealed sequentially

In [ ]:
# ===== RA-UCB Simulation with REAL observations from EdNet KT3 =====

# Load the pseudo-users dataset
pseudo_df = pd.read_csv("./pseudo_users_dataset.csv")

# Get list of questions (arms)
questions = res["question_id"].values
K = len(questions)

# Ground truth parameters (estimated offline) - in SECONDS
p_true = res["p_i"].values
k_vec = res["K_i"].values  # k connu (comme dans le code simulé)
scale_ms = res["scale_i"].values  
scale_s = scale_ms / 10000  # scale en daizaine secondes
lambda_true = 1.0 / scale_s  # λ = 1/scale

# Prepare real observations: organize by question
users = pseudo_df["pseudo_user_id"].unique()
n_users = len(users)

obs_by_question = {}
for q in questions:
    q_data = pseudo_df[pseudo_df["question_id"] == q].copy()
    q_data = q_data.set_index("pseudo_user_id")
    obs_list = []
    for u in users:
        row = q_data.loc[u]
        obs_list.append((row["response_time_ms"] / 1000, row["is_correct"]))  # Convertir en secondes
    obs_by_question[q] = obs_list

# Compute mean budget (average total time per user on 20 questions)
B_all = np.array([sum(obs_by_question[q][u][0] for q in questions) for u in range(n_users)])
B = B_all.mean()  # Budget FIXE = temps moyen

# ===== PARAMETRES DE SIMULATION =====
D = 1
T = 1000  # Rounds per experiment
num_experiments = 1  # 5 experiments with different batches
method = "trust-constr"
n_restarts = 5
# Init phase: at least some rounds per arm for initial estimates
# With T=1000 and K=20, use ~5% of budget for init = 50 rounds = 2-3 per arm
init_steps_per_arm = max(2, int(0.05 * T / K))  # ~2-3 steps par bras

# Uniform allocation (baseline)
x_uniform = np.ones(K) * B / K

print("=" * 60)
print("RA-UCB avec observations RÉELLES (EdNet KT3)")
print(f"5 experiments × T={T} users each")
print("=" * 60)
print(f"K = {K} bras (questions)")
print(f"B = {B:.2f} s (budget fixe = temps moyen)")
print(f"x_uniform = B/K = {B/K:.2f} s par bras")
print(f"D = {D}")
print(f"init_steps_per_arm = {init_steps_per_arm} (init rounds = {init_steps_per_arm * K})")
print(f"n_users disponibles = {n_users}")

print("\nTrue parameters (estimated offline):")
print("p_true =", p_true)
print("lambda_true =", lambda_true)
print("k_vec (connu) =", k_vec)

# Optimal allocation (oracle knows all)
x_star = oracle_weibull_softmax(p_true, lambda_true, lambda_true, B, k_vec, n_restarts=5, seed=0)
print("\nx_star =", x_star)
print("sum(x_star) =", np.sum(x_star))

# Expected reward with uniform allocation
reward_uniform_expected = np.sum(p_true * (1 - np.exp(-(lambda_true * x_uniform) ** k_vec)))
reward_opt_expected = np.sum(p_true * (1 - np.exp(-(lambda_true * x_star) ** k_vec)))
print(f"\nReward espéré oracle: {reward_opt_expected:.4f}")
print(f"Reward espéré uniforme: {reward_uniform_expected:.4f}")
print(f"Regret uniforme par round: {reward_opt_expected - reward_uniform_expected:.4f}")

# Storage for the 5 experiments
all_regrets_ucb = []
all_regrets_uniform = []

# Loop over experiments
for exp in range(num_experiments):
    print(f"\n{'='*40}")
    print(f"Experiment {exp+1}/{num_experiments}")
    print(f"{'='*40}")
    
    # Offset pour utiliser un batch différent de users
    user_offset = exp * T
    if user_offset + T > n_users:
        print(f"WARNING: Not enough users for experiment {exp+1}")
        break
    
    t1 = 0
    user_idx = user_offset

    # Estimation Variables
    n = np.zeros((int(T/K), K))
    Szk = np.zeros((int(T/K), K))
    theta_hat = np.zeros((int(T/K), K))
    lambda_hat = np.zeros((int(T/K), K))
    p_hat = np.zeros((int(T/K), K))
    x_est = np.zeros((int(T/K), K))

    # Reward Variables - UCB
    x_reward = np.zeros((T, K))
    X = np.zeros((T, K))
    Y = np.zeros((T, K))
    F = np.zeros((T, K))
    f = np.zeros((T, K))
    f_star = np.zeros((T, K))
    reward = np.zeros(T)
    opt_reward = np.zeros(T)
    regret = np.zeros(T)
    cum_regret = np.zeros(T)
    
    # Reward Variables - Uniform baseline
    f_uniform = np.zeros((T, K))
    reward_uniform = np.zeros(T)
    regret_uniform = np.zeros(T)

    phi = np.zeros((int(T/K), K))
    psi = np.zeros((int(T/K), K))

    # INITIALIZATION PHASE
    for i in range(K):
        for step in range(init_steps_per_arm):
            if user_idx >= user_offset + T:
                break
                
            x_reward[t1 + 1] = np.zeros(K)
            x_reward[t1 + 1, i] = B
            
            # Observation RÉELLE
            Y_sample = np.zeros(K)
            X_sample = np.zeros(K)
            for k in range(K):
                X_sample[k], is_correct = obs_by_question[questions[k]][user_idx]
                Y_sample[k] = 1 if is_correct else 0
            
            X[t1 + 1] = X_sample
            Y[t1 + 1] = Y_sample
            
            # UCB reward
            for k in range(K):
                if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                    f[t1 + 1, k] = 1
                    F[t1 + 1, k] = X_sample[k]
                else:
                    f[t1 + 1, k] = 0
                    F[t1 + 1, k] = 10 * B
                
                # Oracle reward
                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star[t1 + 1, k] = 1
                else:
                    f_star[t1 + 1, k] = 0
                    
                # Uniform reward
                if Y_sample[k] == 1 and x_uniform[k] >= X_sample[k]:
                    f_uniform[t1 + 1, k] = 1
                else:
                    f_uniform[t1 + 1, k] = 0
            
            phi[t1 // K + 1, i] = f[t1 + 1, i]
            psi[t1 // K + 1, i] = F[t1 + 1, i]
            update_estimates_weibull(t1 // K, i, k_vec[i], phi, psi, n, Szk, theta_hat, lambda_hat, p_hat, x_est, B)
            compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)
            
            # Uniform regret
            reward_uniform[t1 + 1] = np.sum(f_uniform[t1 + 1])
            regret_uniform[t1 + 1] = opt_reward[t1 + 1] - reward_uniform[t1 + 1]
            
            t1 += 1
            user_idx += 1

    # MAIN PHASE
    total_iterations = min((T // K - 1 - t1 // K) * K, T - t1)
    for t in range(t1 // K, T // K - 1):
        for i in range(K):
            if user_idx >= user_offset + T:
                break
            
            # Observation RÉELLE
            Y_sample = np.zeros(K)
            X_sample = np.zeros(K)
            for k in range(K):
                X_sample[k], is_correct = obs_by_question[questions[k]][user_idx]
                Y_sample[k] = 1 if is_correct else 0
            
            # Calcul des bornes de confiance
            p_bar, lambda_bar, lambda_bar_prime = compute_confidence_bounds_weibull_vec(
                t, i, p_hat, lambda_hat, n, x_est, B, D, K, k_vec)
            
            # Sélection UCB
            x_reward[t1 + 1] = select_allocation(lambda_bar, lambda_bar_prime, p_bar, B, k_vec)
            x_est[t + 1, i] = x_reward[t1 + 1, i].copy()
            
            X[t1 + 1] = X_sample
            Y[t1 + 1] = Y_sample
            
            for k in range(K):
                # UCB reward
                if Y_sample[k] == 1 and x_reward[t1 + 1, k] >= X_sample[k]:
                    f[t1 + 1, k] = 1
                    F[t1 + 1, k] = X_sample[k]
                else:
                    f[t1 + 1, k] = 0
                    F[t1 + 1, k] = 10 * B
                
                # Oracle reward
                if Y_sample[k] == 1 and x_star[k] >= X_sample[k]:
                    f_star[t1 + 1, k] = 1
                else:
                    f_star[t1 + 1, k] = 0
                    
                # Uniform reward
                if Y_sample[k] == 1 and x_uniform[k] >= X_sample[k]:
                    f_uniform[t1 + 1, k] = 1
                else:
                    f_uniform[t1 + 1, k] = 0
            
            phi[t + 1, i] = f[t1 + 1, i]
            psi[t + 1, i] = F[t1 + 1, i]
            update_estimates_weibull(t, i, k_vec[i], phi, psi, n, Szk, theta_hat, lambda_hat, p_hat, x_est, B)
            compute_regret(t1, reward, opt_reward, regret, cum_regret, f, f_star)
            
            # Uniform regret
            reward_uniform[t1 + 1] = np.sum(f_uniform[t1 + 1])
            regret_uniform[t1 + 1] = opt_reward[t1 + 1] - reward_uniform[t1 + 1]
            
            t1 += 1
            user_idx += 1
        
        if user_idx >= user_offset + T:
            break
    
    # Store results for this experiment
    cum_regret_ucb = np.cumsum(regret[1:t1+1])
    cum_regret_unif = np.cumsum(regret_uniform[1:t1+1])
    all_regrets_ucb.append(cum_regret_ucb)
    all_regrets_uniform.append(cum_regret_unif)
    
    print(f"  UCB regret final: {cum_regret_ucb[-1]:.2f}")
    print(f"  Uniform regret final: {cum_regret_unif[-1]:.2f}")

print(f"\n{'='*60}")
print("All experiments completed!")
print(f"{'='*60}")

# ===== COMPUTE STATISTICS ACROSS EXPERIMENTS =====
# Pad arrays to same length (in case some experiments ended earlier)
max_len = max(len(r) for r in all_regrets_ucb)
padded_ucb = np.zeros((num_experiments, max_len))
padded_uniform = np.zeros((num_experiments, max_len))

for i in range(num_experiments):
    padded_ucb[i, :len(all_regrets_ucb[i])] = all_regrets_ucb[i]
    padded_ucb[i, len(all_regrets_ucb[i]):] = all_regrets_ucb[i][-1]
    padded_uniform[i, :len(all_regrets_uniform[i])] = all_regrets_uniform[i]
    padded_uniform[i, len(all_regrets_uniform[i]):] = all_regrets_uniform[i][-1]

# Compute mean and confidence intervals (95%)
mean_ucb = np.mean(padded_ucb, axis=0)
std_ucb = np.std(padded_ucb, axis=0)
ci_ucb = 1.96 * std_ucb / np.sqrt(num_experiments)

mean_uniform = np.mean(padded_uniform, axis=0)
std_uniform = np.std(padded_uniform, axis=0)
ci_uniform = 1.96 * std_uniform / np.sqrt(num_experiments)

# ===== PLOT WITH CONFIDENCE INTERVALS =====
fig, ax = plt.subplots(1, 1, figsize=(12, 7))

t_axis = np.arange(1, max_len + 1)

# Plot UCB
ax.plot(t_axis, mean_ucb, 'b-', linewidth=2, label='RA-UCB')
ax.fill_between(t_axis, mean_ucb - ci_ucb, mean_ucb + ci_ucb, alpha=0.3, color='blue')

# Plot Uniform
ax.plot(t_axis, mean_uniform, 'r-', linewidth=2, label='Uniform allocation')
ax.fill_between(t_axis, mean_uniform - ci_uniform, mean_uniform + ci_uniform, alpha=0.3, color='red')

ax.set_xlabel('Round t', fontsize=14)
ax.set_ylabel('Cumulative Regret', fontsize=14)
ax.set_title(f'RA-UCB vs Uniform Allocation (Real Data - {num_experiments} experiments)', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()

# Save to PDF
pdf_path = "./RA_UCB_real_data_results.pdf"
plt.savefig(pdf_path, format='pdf', dpi=300, bbox_inches='tight')
print(f"\nFigure saved to: {pdf_path}")

plt.show()

# ===== SUMMARY STATISTICS =====
print(f"\n{'='*60}")
print(f"Summary ({num_experiments} experiments)")
print(f"{'='*60}")
print(f"\nRA-UCB:")
print(f"  Final regret: {mean_ucb[-1]:.2f} ± {ci_ucb[-1]:.2f}")
print(f"  Std: {std_ucb[-1]:.2f}")

print(f"\nUniform allocation:")
print(f"  Final regret: {mean_uniform[-1]:.2f} ± {ci_uniform[-1]:.2f}")
print(f"  Std: {std_uniform[-1]:.2f}")

print(f"\nImprovement of UCB over Uniform: {(1 - mean_ucb[-1]/mean_uniform[-1])*100:.1f}%")